In [3]:
import pickle
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
ACTIONS = ['UP', 'RIGHT', 'DOWN', 'LEFT', 'WAIT', 'BOMB']
ACTION_TO_INT = {'UP':0, 'RIGHT':1, 'DOWN':2, 'LEFT':3, 'WAIT':4, 'BOMB':5}

In [4]:
# TODO: implement gradient boost
# TODO: check put PyTables to save and load entries of large dictionaries saved as a file

In [5]:
d = dict()
state = np.array([0, 0])
k = tuple(state.tolist())   # convert numpy array to a tuple

d[k] = [[],[]]
d[k][0] = 0
d[k][1] = 1
d[k]

new_state = np.asarray(k)   # convert tuple back to numpy array
new_state

array([0, 0])

In [6]:
with open("my-saved-q-table.pt", "rb") as file:
    q_table:dict = dict(pickle.load(file))
with open("action_vectors.pt", "rb") as file:
    action_vectors:dict = dict(pickle.load(file))

In [7]:
def state_to_action_via_lin_val_est(state_str:str):
    temp_str = state_str[1:-1]
    state = np.fromstring(temp_str, dtype=float, sep=' ')
    Q = np.zeros(len(ACTIONS))
    for i, a in enumerate(ACTIONS):
        Q[i] = np.dot(state, action_vectors[a])
    action_int = Q.argmax()
    return ACTIONS[action_int]

In [8]:
c = 0
for key in q_table.keys():
    #print(key)
    action = q_table[key].argmax()
    print(key, "simple_q_learning: ", ACTIONS[action] )
    print(q_table[key] )
    print()

(1, 1, 3, 1, 1, -1) simple_q_learning:  DOWN
[-2.56663054 -0.4847274   1.30035189 -0.99369456 -9.84715146 -4.14537862]

(1, 0, 3, 2, 1, -1) simple_q_learning:  DOWN
[ 0.26144006 -6.13770968 99.65960943 -7.42751742 -4.49648537 -2.75906005]

(1, 1, -1, 3, 1, -1) simple_q_learning:  LEFT
[ -2.25242937  -1.49687454  -3.65926227   2.28455969 -12.34601115
  -3.94746583]

(1, 0, -1, -2, 1, -1) simple_q_learning:  UP
[ 2.44570118 -2.21928042 -0.02278685 -0.96728004 -0.89039792 -0.82112906]

(1, 1, 2, 1, 1, -1) simple_q_learning:  DOWN
[-1.1040935   0.85249232  1.3708183  -1.58162013 -3.6869916  -1.17246684]

(1, 0, 2, 2, 1, -1) simple_q_learning:  DOWN
[ 0.12522896 -1.17868157  0.75890727 -0.97856772  0.03449617  0.20511229]

(1, 1, 2, 3, 1, -1) simple_q_learning:  RIGHT
[-1.26490535 99.3668719  -1.86087879 -3.09758963 -6.28093251 -2.4528628 ]

(1, 1, 3, 3, 1, -1) simple_q_learning:  RIGHT
[ 1.05496491 42.89807903  1.01179235  4.96981254  1.0952716   5.63626486]

(1, 1, -1, -1, 1, -1) simple_q

In [9]:
print(len(q_table)) 
print(c)

66
0


In [10]:
X = np.eye(3)   # <- samples / features
#X = X.reshape(-1, 1)
y = np.array([1, 2, 3]) # <- tags / Q-val
X

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [11]:
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X,y)
y_pred = regressor.predict([[1, 0, 0]])
y_pred

array([1.])

In [22]:
x_test = np.array([1, 0, 0])
regressor = GradientBoostingRegressor(random_state = 0)
regressor.fit(X,y)
y_pred = regressor.predict([x_test])
y_pred

ValueError: Expected 2D array, got 1D array instead:
array=[1. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [13]:
tuple1 = ((1, 2, 3), (2, 3, 4)) 
print(type(tuple1)) 
print(tuple1) 
print() 
  
# conversion 
array2 = np.array(tuple1) 
print(type(array2)) 
print(array2) 
print() 
  
# list, array and tuple 
array3 = np.array([tuple1]) 
print(type(array3)) 
print(array3)

<class 'tuple'>
((1, 2, 3), (2, 3, 4))

<class 'numpy.ndarray'>
[[1 2 3]
 [2 3 4]]

<class 'numpy.ndarray'>
[[[1 2 3]
  [2 3 4]]]


In [20]:
d = {(1, 1):0, (2,2):2, (3,3):5}
test_list = list(d.keys())
res = np.array(list(map(list, test_list)))
res
#X = list(X)
#np.asarray(X)

array([[1, 1],
       [2, 2],
       [3, 3]])